In [1]:
from IPython.display import Image

In [2]:
Image(url="HASYv2/hasy-data/v2-00010.png")

In [6]:
import csv
from PIL import Image as pil_image
import keras.preprocessing.image

In [7]:
imgs = []
classes = []
with open("HASYv2/hasy-data-labels.csv") as csvfile:
    csvreader = csv.reader(csvfile)
    i = 0
    for row in csvreader:
        if i > 0:
            img = keras.preprocessing.image.img_to_array(pil_image.open("HASYv2/" + row[0]))
            img /= 255.0
            imgs.append((row[0],row[2],img))
            classes.append(row[2])
        i +=1

In [8]:
imgs[0]

('hasy-data/v2-00000.png',
 'A',
 array([[[1., 1., 1.],
         [1., 1., 1.],
         [1., 1., 1.],
         ...,
         [1., 1., 1.],
         [1., 1., 1.],
         [1., 1., 1.]],
 
        [[1., 1., 1.],
         [1., 1., 1.],
         [1., 1., 1.],
         ...,
         [1., 1., 1.],
         [1., 1., 1.],
         [1., 1., 1.]],
 
        [[1., 1., 1.],
         [1., 1., 1.],
         [1., 1., 1.],
         ...,
         [1., 1., 1.],
         [1., 1., 1.],
         [1., 1., 1.]],
 
        ...,
 
        [[1., 1., 1.],
         [1., 1., 1.],
         [1., 1., 1.],
         ...,
         [1., 1., 1.],
         [1., 1., 1.],
         [1., 1., 1.]],
 
        [[1., 1., 1.],
         [1., 1., 1.],
         [1., 1., 1.],
         ...,
         [1., 1., 1.],
         [1., 1., 1.],
         [1., 1., 1.]],
 
        [[1., 1., 1.],
         [1., 1., 1.],
         [1., 1., 1.],
         ...,
         [1., 1., 1.],
         [1., 1., 1.],
         [1., 1., 1.]]], dtype=float32))

In [12]:
len(imgs)

168233

In [13]:
import random
random.shuffle(imgs)
split_idx = int(0.8*len(imgs))
train = imgs[:split_idx]
test = imgs[split_idx:]

In [14]:
import numpy as np

train_input = np.asarray(list(map(lambda row: row[2], train)))
test_input = np.asarray(list(map(lambda row: row[2], test)))

train_output = np.asarray(list(map(lambda row: row[1], train)))
test_output = np.asarray(list(map(lambda row: row[1], test)))

In [15]:
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

In [53]:
# convert class names into one-hot encoding

# first, convert class names into integers
label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(classes)

# then convert integers into one-hot encoding
onehot_encoder = OneHotEncoder(sparse=False)
integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
onehot_encoder.fit(integer_encoded)

# convert train and test output to one-hot
train_output_int = label_encoder.transform(train_output)
train_output = onehot_encoder.transform(train_output_int.reshape(len(train_output_int), 1))
test_output_int = label_encoder.transform(test_output)
test_output = onehot_encoder.transform(test_output_int.reshape(len(test_output_int), 1))

num_classes = len(label_encoder.classes_)
print("Number of classes: %d" % num_classes)

Number of classes: 369


In [16]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D,MaxPooling2D

In [15]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(3,3), activation = 'relu', input_shape=np.shape(train_input[0])))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Conv2D(32, (3,3), activation = 'relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Flatten())
model.add(Dense(1024, activation = 'tanh'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 30, 30, 32)        896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 15, 15, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 13, 13, 32)        9248      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 6, 6, 32)          0         
_________________________________________________________________
flatten (Flatten)            (None, 1152)              0         
_________________________________________________________________
dense (Dense)                (None, 1024)              1180672   
_________________________________________________________________
dropout (Dropout)            (None, 1024)              0

In [17]:
import keras.callbacks
tensorboard = keras.callbacks.TensorBoard(log_dir = '.\logs\mnist-style')

In [17]:
model.fit(train_input, train_output,
         batch_size = 32,
         epochs = 10,
         verbose = 2,
         validation_split = 0.2,
         callbacks = [tensorboard])

Epoch 1/10
3365/3365 - 39s - loss: 1.5302 - accuracy: 0.6296 - val_loss: 1.0023 - val_accuracy: 0.7288
Epoch 2/10
3365/3365 - 38s - loss: 0.9734 - accuracy: 0.7310 - val_loss: 0.9057 - val_accuracy: 0.7520
Epoch 3/10
3365/3365 - 39s - loss: 0.8576 - accuracy: 0.7549 - val_loss: 0.8713 - val_accuracy: 0.7558
Epoch 4/10
3365/3365 - 39s - loss: 0.7904 - accuracy: 0.7688 - val_loss: 0.8690 - val_accuracy: 0.7605
Epoch 5/10
3365/3365 - 38s - loss: 0.7385 - accuracy: 0.7794 - val_loss: 0.8391 - val_accuracy: 0.7680
Epoch 6/10
3365/3365 - 39s - loss: 0.6964 - accuracy: 0.7880 - val_loss: 0.8269 - val_accuracy: 0.7683
Epoch 7/10
3365/3365 - 39s - loss: 0.6561 - accuracy: 0.7968 - val_loss: 0.8527 - val_accuracy: 0.7687
Epoch 8/10
3365/3365 - 39s - loss: 0.6317 - accuracy: 0.8025 - val_loss: 0.8552 - val_accuracy: 0.7674
Epoch 9/10
3365/3365 - 39s - loss: 0.6052 - accuracy: 0.8093 - val_loss: 0.8758 - val_accuracy: 0.7570
Epoch 10/10
3365/3365 - 40s - loss: 0.5842 - accuracy: 0.8132 - val_loss:

In [ ]:
# try various model configurations and parameters to find the best
# warning: it take some time

import time

results = []
for conv2d_count in [1,2]:
    for dense_size in [128,256,512,1024,2048]:
        for dropout in [0.0, 0.25, 0.50, 0.75]:
            model = Sequential()
            for i in range(conv2d_count):
                if i == 0:
                    model.add(Conv2D(32, kernel_size = (3,3), activation = 'relu', input_shape = np.shape(train_input[0])))
                else:
                    model.add(Conv2D(32, kernel_size=(3,3), activation = 'relu'))
                model.add(MaxPooling2D(pool_size = (2,2)))
            model.add(Flatten())
            model.add(Dense(dense_size, activation = 'tanh'))
            if dropout > 0.0:
                model.add(Dropout(dropout))
            model.add(Dense(num_classes, activation = 'softmax'))
            
            model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
            
            log_dir = '.\logs\conv2d_%d-dense_%d-dropout_%.2f' % (conv2d_count, dense_size, dropout)
            tensorboard = keras.callbacks.TensorBoard(log_dir=log_dir)
                              
            start = time.time()
            model.fit(train_input, train_output, batch_size=32, epochs=10, verbose=0, validation_split=0.2, callbacks = [tensorboard])
            score = model.evaluate(test_input, test_output, verbose=2)
            end = time.time()

In [23]:
# rebuild / retrain a model with the best parameters (from the search) and user all data
model = Sequential()
model.add(Conv2D(32, kernel_size = (3,3), activation = 'relu', input_shape=np.shape(train_input[0])))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Conv2D(32, (3,3), activation = 'relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Flatten())
model.add(Dense(128,activation = 'tanh'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation = 'softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
print(model.summary())

# join train and test data so we train the network on all data we have available to us
model.fit(np.concatenate((train_input, test_input)), np.concatenate((train_output, test_output)), batch_size=32, epochs=10, verbose=2)

# save the trained model
model.save("mathsymbols.model")

# save label encoder (to reverse one-hot encoding)
np.save('classes.npy', label_encoder.classes_)

Model: "sequential_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_11 (Conv2D)           (None, 30, 30, 32)        896       
_________________________________________________________________
max_pooling2d_11 (MaxPooling (None, 15, 15, 32)        0         
_________________________________________________________________
conv2d_12 (Conv2D)           (None, 13, 13, 32)        9248      
_________________________________________________________________
max_pooling2d_12 (MaxPooling (None, 6, 6, 32)          0         
_________________________________________________________________
flatten_10 (Flatten)         (None, 1152)              0         
_________________________________________________________________
dense_19 (Dense)             (None, 128)               147584    
_________________________________________________________________
dropout_7 (Dropout)          (None, 128)             

In [20]:
# Load the pre-trained model and predict the math symbol for an arbitary image;
# the code below could be placed in a separate file

import keras.models
model2 = keras.models.load_model("mathsymbols.model")
print(model2.summary())

#restore the class name to integer encoder
label_encoder2 = LabelEncoder()
label_encoder2.classes_ = np.load('classes.npy')

def predict(img_path):
    newimg = keras.preprocessing.image.img_to_array(pil_image.open(img_path))
    newimg /= 255.0
    
    # do the prediction
    prediction = model2.predict(newimg.reshape(1,32,32,3))
    
    #figure out which output neuron had the highest score, and reverse the one-hot encoding
    inverted = label_encoder2.inverse_transform([np.argmax(prediction)]) # argmax finds highest-scoring output
    print("Prediction: %s, confidence: %.2f" % (inverted[0], np.max(prediction)))

Model: "sequential_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_11 (Conv2D)           (None, 30, 30, 32)        896       
_________________________________________________________________
max_pooling2d_11 (MaxPooling (None, 15, 15, 32)        0         
_________________________________________________________________
conv2d_12 (Conv2D)           (None, 13, 13, 32)        9248      
_________________________________________________________________
max_pooling2d_12 (MaxPooling (None, 6, 6, 32)          0         
_________________________________________________________________
flatten_10 (Flatten)         (None, 1152)              0         
_________________________________________________________________
dense_19 (Dense)             (None, 128)               147584    
_________________________________________________________________
dropout_7 (Dropout)          (None, 128)             

In [66]:
# Load the pre-trained model and predict the math symbol for an arbitary image;
# the code below could be placed in a separate file

import keras.models
model2 = keras.models.load_model("mathsymbols2.model")
print(model2.summary())

#restore the class name to integer encoder
label_encoder2 = LabelEncoder()
label_encoder2.classes_ = np.load('classes2.npy')

def predict2(img_path):
    newimg = keras.preprocessing.image.img_to_array(pil_image.open(img_path))
    newimg /= 255.0
    
    # do the prediction
    prediction = model2.predict(newimg.reshape(1,32,32,4))
    
    #figure out which output neuron had the highest score, and reverse the one-hot encoding
    inverted = label_encoder2.inverse_transform([np.argmax(prediction)]) # argmax finds highest-scoring output
    print("Prediction: %s, confidence: %.2f" % (inverted[0], np.max(prediction)))

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 29, 29, 32)        1568      
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 11, 11, 32)        16416     
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 5, 5, 32)          0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 800)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 128)               102528    
_________________________________________________________________
dropout_2 (Dropout)          (None, 128)              

In [48]:
# Example: grab an image (we'll just use a random training image for demonstration purposes)
predict("HASYv2/hasy-data/v2-00010.png")

Prediction: A, confidence: 0.98


# My sample: A B C D E

In [84]:
Image(url="HASYv2/testData/0.png")

In [86]:
Image(url="HASYv2/testData/1.png")

In [87]:
Image(url="HASYv2/testData/2.png")

In [88]:
Image(url="HASYv2/testData/3.png")

In [89]:
Image(url="HASYv2/testData/4.png")

In [85]:
predict("HASYv2/testData/0.png")
predict("HASYv2/testData/1.png")
predict("HASYv2/testData/2.png")
predict("HASYv2/testData/3.png")
predict("HASYv2/testData/4.png")

Prediction: \mathcal{A}, confidence: 0.60
Prediction: \beta, confidence: 0.33
Prediction: \subseteq, confidence: 0.74
Prediction: \mathcal{D}, confidence: 0.72
Prediction: \vDash, confidence: 0.29


# Revisiting the bird species identifier to use images

In [26]:
import numpy as np
from keras.applications.inception_v3 import InceptionV3
from keras.models import Sequential, load_model, Model
from keras.layers import Input, Dropout, Flatten, Conv2D, MaxPooling2D, Dense, Activation, GlobalAveragePooling2D
from keras.optimizers import SGD
from keras.utils import np_utils
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import TensorBoard
import itertools

In [27]:
# all images will be converted to this size
ROWS = 256
COLS = 256
CHANNELS = 3

In [91]:
# split data set
# REF: https://github.com/irfanICMLL/inceptionV2_finetune

from __future__ import absolute_import
from __future__ import print_function
import matplotlib.pyplot as plt
import os
import shutil
from shutil import copyfile

# Change the path to your dataset folder:
base_folder = './CUB_200_2011/'

# These path should be fine
images_txt_path = base_folder+ 'images.txt'
train_test_split_path =  base_folder+ 'train_test_split.txt'
images_path =  base_folder+ 'images/'

# Here declare where you want to place the train/test folders
# You don't need to create them!
test_folder = './test/'
train_folder = './train/'


def ignore_files(dir,files): return [f for f in files if os.path.isfile(os.path.join(dir,f))]

shutil.copytree(images_path,test_folder,ignore=ignore_files)
shutil.copytree(images_path,train_folder,ignore=ignore_files)

with open(images_txt_path) as f:
  images_lines = f.readlines()

with open(train_test_split_path) as f:
  split_lines = f.readlines()

test_images, train_images = 0,0

for image_line,split_line in zip(images_lines,split_lines):

  image_line = (image_line.strip()).split(' ')
  split_line = (split_line.strip()).split(' ')

  image = plt.imread(images_path + image_line[1])

  # Use only RGB images, avoid grayscale
  if len(image.shape) == 3:

    # If test image
    if(int(split_line[1]) is 0):
      copyfile(images_path+image_line[1],test_folder+image_line[1])
      test_images += 1 
    else:
      # If train image
      copyfile(images_path+image_line[1],train_folder+image_line[1])
      train_images += 1 

print(train_images,test_images)
assert train_images == 5990
assert test_images == 5790

print('Dataset succesfully splitted!')

5990 5790
Dataset succesfully splitted!


In [93]:
train_image_generator = ImageDataGenerator(horizontal_flip=True, rescale=1./255, rotation_range=45)
test_image_generator = ImageDataGenerator(horizontal_flip=False, rescale=1./255, rotation_range=0)

train_generator = train_image_generator.flow_from_directory('train', target_size=(ROWS, COLS), class_mode='categorical')
test_generator = test_image_generator.flow_from_directory('test', target_size=(ROWS, COLS), class_mode='categorical')

Found 5990 images belonging to 200 classes.
Found 5790 images belonging to 200 classes.


In [94]:
train_generator.reset()
test_generator.reset()

model = Sequential()
model.add(Conv2D(64, (3,3), input_shape=(ROWS, COLS, CHANNELS)))
model.add(Activation('relu'))
model.add(Conv2D(64, (3,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(4,4)))
model.add(Conv2D(64, (3,3)))
model.add(Activation('relu'))
model.add(Conv2D(64, (3,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(4,4)))
model.add(Flatten())
model.add(Dropout(0.5))
model.add(Dense(400))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(200))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adamax', metrics=['accuracy'])

model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_8 (Conv2D)            (None, 254, 254, 64)      1792      
_________________________________________________________________
activation (Activation)      (None, 254, 254, 64)      0         
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 252, 252, 64)      36928     
_________________________________________________________________
activation_1 (Activation)    (None, 252, 252, 64)      0         
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 63, 63, 64)        0         
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 61, 61, 64)        36928     
_________________________________________________________________
activation_2 (Activation)    (None, 61, 61, 64)       

In [102]:
tensorboard = TensorBoard(log_dir='./logs/custom')

model.fit_generator(train_generator, steps_per_epoch=128, epochs=10, callbacks=[tensorboard], verbose=1)

Epoch 1/10
128/128 [==============================] - 310s 2s/step - loss: 5.1952 - accuracy: 0.0108
Epoch 2/10
128/128 [==============================] - 311s 2s/step - loss: 5.1277 - accuracy: 0.0118
Epoch 3/10
128/128 [==============================] - 314s 2s/step - loss: 5.0402 - accuracy: 0.0187
Epoch 4/10
128/128 [==============================] - 318s 2s/step - loss: 4.9782 - accuracy: 0.0214
Epoch 5/10
128/128 [==============================] - 316s 2s/step - loss: 4.9265 - accuracy: 0.0265
Epoch 6/10
128/128 [==============================] - 312s 2s/step - loss: 4.8500 - accuracy: 0.0295
Epoch 7/10
128/128 [==============================] - 318s 2s/step - loss: 4.7729 - accuracy: 0.0344
Epoch 8/10
128/128 [==============================] - 312s 2s/step - loss: 4.7011 - accuracy: 0.0391
Epoch 9/10
128/128 [==============================] - 313s 2s/step - loss: 4.6532 - accuracy: 0.0523
Epoch 10/10
128/128 [==============================] - 311s 2s/step - loss: 4.5665 - accura

In [ ]:
print(model.evaluate_generator(test_generator, steps=1000))

In [109]:
# create the base pre-trained model
base_model = InceptionV3(weights='imagenet', include_top=False)

# add a global spatial average pooling layer
x = base_model.output
x = GlobalAveragePooling2D()(x)
# add a fully-connected layer
x = Dense(1024, activation='relu')(x)
out_layer = Dense(200, activation='softmax')(x)

# this is the model we will train
model = Model(inputs=base_model.input, outputs=out_layer)

In [110]:
# first: train only the top layers (which were randomly initialized)
# i.e. freeze all convolutional InceptionV3 layers
for layer in base_model.layers:
    layer.trainable = False

model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, None, None,  0                                            
__________________________________________________________________________________________________
conv2d_106 (Conv2D)             (None, None, None, 3 864         input_2[0][0]                    
__________________________________________________________________________________________________
batch_normalization_94 (BatchNo (None, None, None, 3 96          conv2d_106[0][0]                 
__________________________________________________________________________________________________
activation_100 (Activation)     (None, None, None, 3 0           batch_normalization_94[0][0]     
____________________________________________________________________________________________

In [111]:
tensorboard = TensorBoard(log_dir='./logs/inceptionv3')

model.fit_generator(train_generator, steps_per_epoch=32, epochs=100, callbacks=[tensorboard], verbose=1)

Epoch 1/100
32/32 [==============================] - 26s 721ms/step - loss: 6.9253 - accuracy: 0.0180
Epoch 2/100
32/32 [==============================] - 23s 707ms/step - loss: 4.8185 - accuracy: 0.0749
Epoch 3/100
32/32 [==============================] - 22s 688ms/step - loss: 4.1948 - accuracy: 0.0939
Epoch 4/100
32/32 [==============================] - 22s 695ms/step - loss: 3.6844 - accuracy: 0.1573
Epoch 5/100
32/32 [==============================] - 22s 688ms/step - loss: 3.2513 - accuracy: 0.2328
Epoch 6/100
32/32 [==============================] - 22s 692ms/step - loss: 3.1214 - accuracy: 0.2189
Epoch 7/100
32/32 [==============================] - 22s 691ms/step - loss: 3.0374 - accuracy: 0.2658
Epoch 8/100
32/32 [==============================] - 22s 680ms/step - loss: 2.7920 - accuracy: 0.2961
Epoch 9/100
32/32 [==============================] - 22s 684ms/step - loss: 2.7428 - accuracy: 0.2783
Epoch 10/100
32/32 [==============================] - 22s 686ms/step - loss: 2.457

32/32 [==============================] - 22s 691ms/step - loss: 0.8878 - accuracy: 0.7318
Epoch 81/100
32/32 [==============================] - 23s 695ms/step - loss: 1.0168 - accuracy: 0.6891
Epoch 82/100
32/32 [==============================] - 23s 696ms/step - loss: 0.9367 - accuracy: 0.7182
Epoch 83/100
32/32 [==============================] - 23s 697ms/step - loss: 0.9752 - accuracy: 0.7053
Epoch 84/100
32/32 [==============================] - 23s 712ms/step - loss: 0.9288 - accuracy: 0.7279
Epoch 85/100
32/32 [==============================] - 23s 695ms/step - loss: 1.0419 - accuracy: 0.6888
Epoch 86/100
32/32 [==============================] - 23s 696ms/step - loss: 0.9521 - accuracy: 0.7297
Epoch 87/100
32/32 [==============================] - 22s 693ms/step - loss: 1.0042 - accuracy: 0.7153
Epoch 88/100
32/32 [==============================] - 22s 691ms/step - loss: 1.0619 - accuracy: 0.7037
Epoch 89/100
32/32 [==============================] - 22s 691ms/step - loss: 1.0143 - 

In [ ]:
print(model.evaluate_generator(test_generator, steps=5000))

In [112]:
# unfreeze all layers for more training
for layer in model.layers:
    layer.trainable = True

# we need to recompile the model for these modifications to take effect
# we use SGD with a low learning rate
model.compile(optimizer=SGD(lr=0.0001, momentum=0.9), loss='categorical_crossentropy', metrics=['accuracy'])

model.fit_generator(train_generator, steps_per_epoch=32, epochs=100)

Epoch 1/100
32/32 [==============================] - 138s 4s/step - loss: 2.6111 - accuracy: 0.4017
Epoch 2/100
32/32 [==============================] - 134s 4s/step - loss: 2.0951 - accuracy: 0.4869
Epoch 3/100
32/32 [==============================] - 134s 4s/step - loss: 1.5582 - accuracy: 0.5873
Epoch 4/100
32/32 [==============================] - 134s 4s/step - loss: 1.3347 - accuracy: 0.6049
Epoch 5/100
32/32 [==============================] - 134s 4s/step - loss: 1.2109 - accuracy: 0.6470
Epoch 6/100
32/32 [==============================] - 135s 4s/step - loss: 1.3775 - accuracy: 0.6117
Epoch 7/100
32/32 [==============================] - 136s 4s/step - loss: 1.0367 - accuracy: 0.6911
Epoch 8/100
32/32 [==============================] - 137s 4s/step - loss: 1.0245 - accuracy: 0.6948
Epoch 9/100
32/32 [==============================] - 131s 4s/step - loss: 0.9376 - accuracy: 0.7233
Epoch 10/100
32/32 [==============================] - 135s 4s/step - loss: 0.9905 - accuracy: 0.7115

32/32 [==============================] - 130s 4s/step - loss: 0.3202 - accuracy: 0.9129
Epoch 83/100
32/32 [==============================] - 127s 4s/step - loss: 0.2885 - accuracy: 0.9145
Epoch 84/100
32/32 [==============================] - 129s 4s/step - loss: 0.3256 - accuracy: 0.9029
Epoch 85/100
32/32 [==============================] - 129s 4s/step - loss: 0.2877 - accuracy: 0.9301
Epoch 86/100
32/32 [==============================] - 130s 4s/step - loss: 0.2997 - accuracy: 0.9096
Epoch 87/100
32/32 [==============================] - 129s 4s/step - loss: 0.2710 - accuracy: 0.9294
Epoch 88/100
32/32 [==============================] - 131s 4s/step - loss: 0.2738 - accuracy: 0.9248
Epoch 89/100
32/32 [==============================] - 132s 4s/step - loss: 0.2489 - accuracy: 0.9307
Epoch 90/100
32/32 [==============================] - 128s 4s/step - loss: 0.2906 - accuracy: 0.9084
Epoch 91/100
32/32 [==============================] - 131s 4s/step - loss: 0.2730 - accuracy: 0.9281
Epo

In [ ]:
test_generator.reset()
print(model.evaluate_generator(test_generator, steps=5000))

In [113]:
model.save("birds-inceptionv3.model")

INFO:tensorflow:Assets written to: birds-inceptionv3.model\assets


In [114]:
from keras.models import load_model
from keras.preprocessing import image
from os import listdir
import numpy as np

ROWS = 256
COLS = 256

In [116]:
CLASS_NAMES = sorted(listdir('train'))
model = load_model('birds-inceptionv3.model')

In [117]:
def predict(fname):
    img = image.load_img(fname, target_size=(ROWS, COLS))
    img_tensor = image.img_to_array(img) # (height, width, channels)
    # (1, height, width, channels), add a dimension because the model expects this shape:
    # (batch_size, height, width, channels)
    img_tensor = np.expand_dims(img_tensor, axis=0) 
    img_tensor /= 255. # model expects values in the range [0, 1]
    prediction = model.predict(img_tensor)[0]
    best_score_index = np.argmax(prediction)
    bird = CLASS_NAMES[best_score_index] # retrieve original class name
    print("Prediction: %s (%.2f%%)" % (bird, 100*prediction[best_score_index]))

In [125]:
predict('testBirds/annas_hummingbird_sim_1.jpg')

Prediction: 067.Anna_Hummingbird (86.54%)


In [128]:
predict('testBirds/house_wren.jpg')

Prediction: 198.Rock_Wren (99.63%)


In [129]:
predict('testBirds/canada_goose_1.jpg')

Prediction: 086.Pacific_Loon (49.33%)
